In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gensim
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools



from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

#import researchpy as rp
import scipy.stats as stats

from sklearn.metrics import confusion_matrix
import seaborn as sns

from nltk.stem.snowball import SnowballStemmer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
%%time

# Load 30 Word2Vec models of MPs in T1 & T2

dictOfModels = {}
folderPath = '/kaggle/input/stemmedpartymodels/kaggle/working/models-by-party-and-time'
for file in os.listdir(folderPath):
    filePath = folderPath + '/' + file
    print(filePath)
    if(len(filePath.split('.'))>1):
        continue
    else:
        model = gensim.models.Word2Vec.load(filePath)
        dictOfModels[file] = model

In [30]:
change =   ['exiting', 'seaborne', 'eurotunnel', 'withdrawal', 'departures', 'unicorn', 'remainers', 'exit', 'surrender', 'departure',
     'triggering', 'stockpiling', 'locks', 'expulsion', 'blindfold', 'cliff', 'exits', 'triggered', 'brexiteer', 'soft', 'plus',
     'trigger', 'backroom', 'attribute', 'invoked', 'brexit', 'edge', 'canary', 'unicorns', 'withdrawing', 'wharton', 'invoking', 
     'withdrawn', 'brexiteers', 'fanatics', 'postponement', 'juncture', 'currencies', 'currency', 'operability', 'operable', 'leavers',
     'invoke', 'article', 'eurozone', 'cluelessness', 'clueless', 'surrendered', 'cake', 'red', 'euroscepticism', 'prorogation', 'gove',
     'whittingdale', 'norway', 'deflationary', 'moribund', 'eurosceptic', 'deutschmark', 'courting', 'inject', 'deal', 'withdraw',
     'withdrawals', 'eurosceptics', 'surrendering', 'leaving', 'signifying', 'inflationary', 'ceded', 'absentia', 'treachery', 'dollar',
     'canada', 'pragmatist', 'oven', 'ready', 'brexiters', 'control', 'capitulation', 'leave', 'referendum', 'agreement', 'prorogue',
     'smoothest', 'depreciate', 'managed', 'mutiny', 'ideologues', 'foreign', 'eec', 'war', 'prorogued', 'hannan', 'appease', 'pendolino', 
     'southbound', 'jailed', 'left', 'line', 'hard', 'bill']

no_change = ['prime', 'even', 'parliament', 'mr', 'eu', 'bill', 'future', 'care', 'well', 'constituency', 'tax', 'children', 'uk',
             'business', 'european', 'report', 'case', 'sure', 'like', 'see', 'state', 'health', 'order', 'back', 'new', 'hope', 'local',
             'national', 'country', 'secretary', 'public', 'right', 'much', 'say', 'first', 'minister', 'labour', 'look', 'system', 'whether', 
             'members', 'million', 'good', 'today', 'services', 'clear', 'help', 'time', 'place', 'put', 'last', 'must', 'money', 'one', 
             'way', 'friend', 'work', 'would', 'think', 'two', 'great', 'could', 'lady', 'us', 'come', 'however', 'may', 'going', 'go',
             'given', 'need', 'year', 'debate', 'might', 'part', 'get', 'want', 'make', 'point', 'committee', 'years', 'also', 'know',
             'government', 'take', 'house', 'agree', 'member', 'number', 'across', 'made', 'give', 'gentleman', 'important', 'said',
             'people', 'issue', 'support', 'ensure']




'''change = ['exiting', 'seaborne', 'eurotunnel', 'withdrawal', 'departures', 'unicorn', 'remainers', 'exit', 'surrender',
          'departure', 'triggering', 'stockpiling', 'expulsion', 'blindfold', 'cliff', 'lighter', 'exits', 'triggered',
          'brexiteer', 'soft', 'plus', 'trigger', 'backroom', 'invoked', 'protesting', 'brexit', 'edge', 'canary', 
          'unicorns', 'withdrawing', 'invoking', 'withdrawn', 'manor', 'brexiteers', 'fanatics', 'postponement', 
          'currencies', 'currency', 'operability', 'operable', 'leavers', 'invoke', 'article', 'eurozone', 'clueless',
          'surrendered', 'cake', 'red', 'euroscepticism', 'prorogation', 'lining', 'gove', 'norway', 'deflationary',
          'moribund', 'eurosceptic', 'deutschmark', 'courting', 'deal', 'withdraw', 'dab', 'withdrawals', 'eurosceptics',
          'surrendering', 'aldous', 'lanarkshire', 'leaving', 'signifying', 'roofs', 'ceded', 'absentia', 'treachery',
          'dollar', 'canada', 'pragmatist', 'oven', 'ready', 'brexiters', 'control', 'capitulation', 'leave', 'referendum',
          'agreement', 'prorogue', 'smoothest', 'depreciate', 'managed', 'mutiny', 'overvalued', 'ideologues', 'foreign',
          'eec', 'war', 'prorogued', 'hannan', 'appease', 'pendolino', 'southbound', 'left', 'line', 'hard', 'bill']



no_change = ['prime', 'even', 'parliament', 'mr', 'eu', 'bill', 'future', 'care', 'well',
             'constituency', 'tax',
             'children', 'uk', 'business']

no_change = ['prime', 'even', 'parliament', 'mr', 'eu', 'bill', 'future', 'care', 'well', 'constituency', 'tax',
             'children', 'uk', 'business', 'european', 'report', 'case', 'sure', 'like', 'see', 'state', 'health',
             'order', 'back', 'new', 'hope', 'local', 'national', 'country', 'secretary', 'public', 'right', 'much',
             'say', 'first', 'minister', 'labour', 'look', 'system', 'whether', 'members', 'million', 'good', 'today', 
             'services', 'clear', 'help', 'time', 'place', 'put', 'last', 'must', 'money', 'one', 'way', 'friend',
             'work', 'would', 'think', 'two', 'great', 'could', 'lady', 'us', 'come', 'however', 'may', 'going', 'go',
             'given', 'need', 'year', 'debate', 'might', 'part', 'get', 'want', 'make', 'point', 'committee', 'years',
             'also', 'know', 'government', 'take', 'house', 'agree', 'member', 'number', 'across', 'made', 'give', 
             'gentleman', 'important', 'said', 'people', 'issue', 'support', 'ensure']'''


In [ ]:
# Stem change and no change

from nltk.stem.snowball import SnowballStemmer


In [58]:

stemmer = SnowballStemmer("english")
change= list(set([stemmer.stem(y) for y in change]))
no_change= list(set([stemmer.stem(y) for y in no_change]))

#print(len(change),len(no_change))
print(len(set(change)), len(set(no_change)))
      

In [59]:
no_change

In [60]:
len(dictOfModels['df_t1_Conservative'].wv.index_to_key)

In [61]:
#To see party wise similar words, analysis only
'''dictOfModels
for i in dictOfModels.keys():
  print(i, dictOfModels[i], dictOfModels[i].wv.similar_by_word('government', 10))
  print('- x - x - x - x - x - x - x - x - x')'''

In [62]:
dictOfModels.keys()

In [63]:
# Slightly modified to now find the cosine difference between provided vectors instead of
# fetching vectors from known models 
def cosine_similarity(vec1, vec2):
  sc = 1-spatial.distance.cosine(vec1, vec2)
  return sc


cosine_similarity_df = pd.DataFrame(columns = ('Word', 'Cosine_similarity'))

In [64]:
def computeAvgVec(mKeys, w):
    #print(mKeys)
    if(w in dictOfModels[mKeys[0]].wv.index_to_key):
        
        modelsSum = np.zeros(dictOfModels[mKeys[0]].layer1_size)

        for k in mKeys:
            #Something like this to refer to t's vector
            #vector = dictOfModels[k].wv.get_index(t)#
            #print(k)
            vectorPerModel = dictOfModels[k].wv[w]
            modelsSum = np.add(modelsSum, vectorPerModel)

        avgEmbedding =np.divide(modelsSum, len(mKeys))
        return avgEmbedding
    else:
        print('Word '+str(w) + ' not found in models vocab')
        return []

In [65]:
%%time
words_of_interest = change + no_change

t1Keys = [k for k in dictOfModels.keys() if 'df_t1' in k] # 32 keys
t2Keys = [k for k in dictOfModels.keys() if 'df_t2' in k] # 26 keys
#t2Keys

In [66]:
%%time
# Compute average of word in T1 and in T2 and store average vectors and cosine difference   
for word in words_of_interest:
        print(word)
        #Provide a list of keys to average computation model for it to
        #compute average vector amongst these models
        
        avgVecT1 = computeAvgVec(t1Keys, word)
        avgVecT2 = computeAvgVec(t2Keys, word)
        
        if(avgVecT1==[] or avgVecT2==[]):
            
            print(str(word) + ' Word not found')
            continue
            
        else:
            
            # Cos similarity between averages
            cosSimilarity = cosine_similarity(avgVecT1, avgVecT2)
            #avgVecRow = {}
            #Append row 
            #cosine_similarity_df = pd.concat(cosine_similarity_df, {'Word': word, 'Cosine_similarity': cosSimilarity})
            
            cosine_similarity_df = cosine_similarity_df.append({'Word': word, 'Cosine_similarity': cosSimilarity}, ignore_index=True)
           


In [67]:
cosine_similarity_df

In [68]:
words_of_interest = cosine_similarity_df[cosine_similarity_df['Word'].isin(change+no_change)]

words_of_interest.loc[words_of_interest['Word'].isin(change), 'semanticDifference'] = 'change' 
words_of_interest.loc[words_of_interest['Word'].isin(no_change), 'semanticDifference'] = 'no_change' 

In [69]:
words_of_interest.sort_values(by='Cosine_similarity')['semanticDifference'].value_counts()

In [70]:
%%time
cosine_similarity_df_sorted = cosine_similarity_df.sort_values(by='Cosine_similarity', ascending=True)
cosine_similarity_df_sorted

In [71]:
words_of_interest = cosine_similarity_df_sorted[cosine_similarity_df_sorted['Word'].isin(change+no_change)]

words_of_interest.loc[words_of_interest['Word'].isin(change), 'semanticDifference'] = 'change' 
words_of_interest.loc[words_of_interest['Word'].isin(no_change), 'semanticDifference'] = 'no_change' 

In [72]:
words_of_interest['semanticDifference'].value_counts()

In [73]:
words_of_interest[words_of_interest['semanticDifference']=='no_change']

In [74]:
words_of_interest[words_of_interest['semanticDifference']=='no_change'].Cosine_similarity.describe()

In [75]:
words_of_interest[words_of_interest['semanticDifference']=='change'].shape

In [76]:
words_of_interest.head(30)

In [77]:
words_of_interest.tail(20)

In [78]:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
import seaborn as sns


In [79]:
X = words_of_interest['Cosine_similarity'].values.reshape(-1,1)
y = words_of_interest['semanticDifference']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)


In [80]:
%%time 
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [81]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
print('Accuracy of logistic regression classifier on test set(Metrics Accuracy): {:.2f}'.format(metrics.accuracy_score(y_test, y_pred)))
print('Precision of logistic regression classifier on test set: {:.2f}'.format(metrics.precision_score(y_test, y_pred,pos_label='change')))
print('Recall of logistic regression classifier on test set: {:.2f}'.format(metrics.recall_score(y_test, y_pred, pos_label='change')))
print('F1 Score of logistic regression classifier on test set: {:.2f}'.format(metrics.f1_score(y_test, y_pred,pos_label='change')))

In [82]:
cf = confusion_matrix(y_test, y_pred)
print(cf)
ax = sns.heatmap(cf, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['change','no_change'])
ax.yaxis.set_ticklabels(['change','no_change'])

## Display the visualization of the Confusion Matrix.
plt.show()

# Nearest Neighbours' Evaluation

In [83]:
%%time 
#words_of_interest[words_of_interest['semanticDifference']=='change']['Word'].wv.similar_by_word()
#brexitNeighbours = model1.wv.similar_by_word('brexit', 100)

#neighboursInT1 = []
#neighboursInT2 = []
overlappingNeighbours = []

for en, word in enumerate(words_of_interest['Word']):
#words_of_interest['Word'].to_list():

    #print(0)
    for sen, md in enumerate(dictOfModels.values()):
        x = md.wv.similar_by_word(word,100) 
        x = [tup[0] for tup in x]
        if(sen==0):
            overlappingNeighbours.append(x)
        else:
            overlap = set(overlappingNeighbours[en]).intersection(x)
            overlappingNeighbours[en]=overlap
        
    
words_of_interest['overlappingNeighbours'] = list(overlappingNeighbours)

words_of_interest['overlappingNeighbours'] = list(overlappingNeighbours)
words_of_interest['overlapCount'] = words_of_interest['overlappingNeighbours'].map(len)


#words_of_interest['neighboursInT2'] = neighboursInT2

#words_of_interest['overlappingNeighbours'] = ?
#intersectingNeighbs = set(words_of_interest['neighboursInT1'].to_list()).intersect(words_of_interest['neighboursInT2'].to_list())


'''lengthOverlap = []

for index in (words_of_interest['neighboursInT1'].index):
    neighboursT1 = words_of_interest.at[index, 'neighboursInT1']
    neighboursT2 = words_of_interest.at[index, 'neighboursInT2']
    lengthOverlap.append(len(set(neighboursT1).intersection(neighboursT2)))

words_of_interest['overlappingNeighbours'] = lengthOverlap'''

In [84]:
words_of_interest

In [85]:
X = words_of_interest['overlapCount'].values.reshape(-1,1)
y = words_of_interest['semanticDifference']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6)


In [86]:
%%time 
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Accuracy of logistic regression classifier on test set(Metrics Accuracy): {:.2f}'.format(metrics.accuracy_score(y_test, y_pred)))
print('Precision of logistic regression classifier on test set: {:.2f}'.format(metrics.precision_score(y_test, y_pred,pos_label='change')))
print('Recall of logistic regression classifier on test set: {:.2f}'.format(metrics.recall_score(y_test, y_pred, pos_label='change')))
print('F1 Score of logistic regression classifier on test set: {:.2f}'.format(metrics.f1_score(y_test, y_pred,pos_label='change')))

cf_neighb_100 = confusion_matrix(y_test, y_pred)

In [87]:
ax = sns.heatmap(cf_neighb_100, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['change','no_change'])
ax.yaxis.set_ticklabels(['change','no_change'])

## Display the visualization of the Confusion Matrix.
plt.show()